## LightGBM
### LightGBM 장단점
  - XGB보다도 학습에 걸리는 시간이 훨씬 적으며 <span style ='color:lime'>메모리 사용량도 상대적으로 적다.</span>
  - <u>카테고리형 피처를 자동 변환</u>하고, 예측 성능 역시 큰 차이가 없다.
  - <span style ='color:lime'>**데이터의 갯수가 적을 경우 과적합**</span>이 발생하기 쉽다.
  - 적은 데이터의 갯수에 대한 기준은 애매하지만 <u>LightGBM의 공식 문서에서 10,000건 이하</u>라고 기술하였다.
 <br>
 <br>
 
### 트리 기반 알고리즘 특징
  - 기존의 대부분 트리 기반 알고리즘은 트리의 깊이를 효과적으로 줄이기 위한 <span style ='color:lime'>균형 트리 분할(Level Wise) 방식</span>을 사용한다.
  - 최대한 균형 잡힌 트리를 유지하면서 분할하기 때문에 <span style ='color:lime'>깊이가 최소화 되며 오버피팅에 보다 강한 구조</sapn>를 가진다.
  - 하지만 <u>균형을 맞추기 위한 시간이 오래 걸리는 단점</u>이 있다.
<br>
<br>

### LightGBM 특징
  - LightGBM은 일반 GBM 계열의 트리 분할 방법과 다르게 <u>리프 중심 트리 분할(Leaf Wise) 방식을 사용</u>한다.
  - 트리의 균형을 맞추지 않고 최대 손실 값(max delta loss)을 가지는 리프 노드를 지속적으로 분할하여 <span style ='color:lime'>깊이가 증가하고 비대칭적인 트리를 생성</span>한다.
  - 이렇게 생성된 트리는 학습을 반복할수록 결국은 균형 트리 분할 방식보다 <u>예측 오류 손실을 최소화 할 수 있다는 것이 LightGBM 구현사상</u>이다.

## 필수라이브러리

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from IPython.display import Image # 이미지 주소로 불러오는 

In [2]:
# 매직 명령어
%matplotlib inline
%config InlinedBackend.figure_format = 'retina'

mpl.rc('font', family='batang')
mpl.rcParams['font.family']='batang'
mpl.rc('axes', unicode_minus=False)

sns.set_style('dark') # 노트
sns.set(color_codes=True)
plt.rc('figure', figsize=(10,8))

warnings.filterwarnings('ignore')

## 데이터 로딩 및 세트 분리

In [3]:
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data, target = load_breast_cancer(True) # 데이터와 타깃값 분리해서 로딩하기

print(data.shape, target.shape) # 확인

(569, 30) (569,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,
                                                   random_state=156)

evals = [(X_test, y_test)] 

#학습
lgbm = LGBMClassifier(n_estimators = 4000)
lgbm.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss',
        eval_set = evals, verbose = True) # xgboost 속성 참고

[1]	valid_0's binary_logloss: 0.565079
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.507451
[3]	valid_0's binary_logloss: 0.458489
[4]	valid_0's binary_logloss: 0.417481
[5]	valid_0's binary_logloss: 0.385507
[6]	valid_0's binary_logloss: 0.355773
[7]	valid_0's binary_logloss: 0.329587
[8]	valid_0's binary_logloss: 0.308478
[9]	valid_0's binary_logloss: 0.285395
[10]	valid_0's binary_logloss: 0.267055
[11]	valid_0's binary_logloss: 0.252013
[12]	valid_0's binary_logloss: 0.237018
[13]	valid_0's binary_logloss: 0.224756
[14]	valid_0's binary_logloss: 0.213383
[15]	valid_0's binary_logloss: 0.203058
[16]	valid_0's binary_logloss: 0.194015
[17]	valid_0's binary_logloss: 0.186412
[18]	valid_0's binary_logloss: 0.179108
[19]	valid_0's binary_logloss: 0.174004
[20]	valid_0's binary_logloss: 0.167155
[21]	valid_0's binary_logloss: 0.162494
[22]	valid_0's binary_logloss: 0.156886
[23]	valid_0's binary_logloss: 0.152855
[24]	valid_0's binary_loglo

LGBMClassifier(n_estimators=4000)

### 예측과 평가

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

predict = lgbm.predict(X_test) # 예측

In [8]:
# 정확도
accuracy = accuracy_score(y_test, predict) 
accuracy

0.956140350877193

In [9]:
# 정밀도 
preci = precision_score(y_test, predict)
preci

0.95

In [10]:
# 재현율
recall = recall_score(y_test, predict)
recall

0.987012987012987

In [11]:
# F1
f1 = f1_score(y_test, predict)
f1

0.9681528662420381

In [ ]:
# 예측
predict_proba_po = lgbm.predict_proba(X_test)[:,1].reshape(-1,1) # 양성값만

In [ ]:
# auc 커브
auc = roc_auc_score(y_test, predict_proba_po)

In [ ]:
# 혼돈 행렬
confu = confusion_matrix(y_test, predict_proba_po)